Universidade Federal de Alagoas

Instituto de Computa��o


Disciplina: Sistemas Distribu�dos

Semestre letivo: 2019.1

Professor: Andr� Lage Freitas


# AB2 - Reavalia��o

**Importante**. Leia atentamente as especifica��es pois s� ser�o aceitos trabalhos que as obedecerem.


## Parte 1: Implementa��o de um monitor de carga de trabalho descentralizado

**Implementar uma vers�o modificada do detector de falha n�o confi�vel** (_unreliable failure detector_) descrito na Se��o _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modifica��o** consiste em enviar informa��es sobre a carga do sistema (_system workload_) em vez de informa��es sobre falhas que indicariam os processos suspeitos e n�o-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviar�o a medi��o interna da sua carga de trabalho. 

Para simular a medi��o da carga de trabalho dos processos, utilize n�meros aleat�rios entre `0` e `1` com precis�o de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como per�odo de envio das mensagens `2` segundos.

### Detalhes de implementa��o

Cada processo do sistema dever� ser implementado como um Julia Worker. O Master ser� apenas o gestor do experimento, ou seja, ser� respons�vel para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo � descentralizado.

Para fins de visualiza��o do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A �ltima linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 s�o `0.4,0.9,0.7`, ou seja, os Workers t�m as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master � 1 e os Workers t�m ID maiores que 1.

## Parte 2: Implementa��o de um algoritmo de Elei��o

**Implementar o algoritmo de Elei��o** baseado em anel (_ring-based election algorithm_) descrito na Se��o _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condi��es.

1. Uma elei��o nova � disparada **quando a carga de trabalho do l�der for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do l�der tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a elei��o.
1. Caso n�o haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um l�der for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programa��o distribu�da em Julia

Para utilizar o suporte de programa��o distribu�da em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (n�o extensiva) de fun��es que poder�o ajudar na programa��o distribu�da para a implementa��o do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma r�pida ajuda sobre as fun��es, utilize `?` na frente da fun��o (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documenta��o oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais d�vidas.

## C�digo-fonte

Documente seu c�digo em coment�rios no pr�prio c�digo-fonte.

**Todo o seu c�digo tem que estar colado na pr�xima c�lula**, mesmo se ele tenha sido feito em IDEs, c�lulas separadas, etc. O Professor testar� a apenas o que estiver colado na pr�xima c�lula.

In [1]:
# Espa�o para escrever o c�digo fonte.
using Pkg
Pkg.add("Distributed")
using Distributed

#TODO

# workers
addprocs(3)
# work loads
@everywhere workloads = zeros(1,nworkers())
# current leader
@everywhere leaderId = 2

@everywhere function election(workerId, workerwld)
    myid = myid()
    currwld = workloads[myid-1]
    nextId = myid+1

    if myid == workerId && myid == leaderId
        for i in workers()    
            
            csId[i-1] = -1
            cswld[i-1] = 1.0
        end
        leaderId = newLeaderId
        csId[myid-1] = -1
        cswld[myid-1] = 1.0
        println("New leader elected! Worker $(leaderId)")
    else
        # If the worker isnt the current leader:
        if myid != leaderId
            if myWorkload < workerwld
                cswld[myid-1] = myWorkload
            elseif myWorkload > workerwld
                cswld[myid-1] = workerwld
            else
                if myid > workerId
                    csId[myid-1] = myid
                    cswld[myid-1] = myWorkload
                else
                
                    cswld[myid-1] = workerwld
                end
            end
        else
            if myWorkload >= 0.8
                
                cswld[myid-1] = workerwld
            else
                cswld[myid-1] = myWorkload
            end
        end
        if nextId > length(workers())+1
            nextId = 2
        end
        @spawnat nextId election(nextId, cswld[myid-1])   
    end
end



@everywhere function updateWorkloads(workerId, workerwld)
    myid = myid()
    
   # if csId[myid-1] == -1    
    workloads[workerId-1] = workerwld
        #init election when the load came from the leader and is greater than or equal  0.8
    println(" $(workerId) e $(workloads[workerId-1])")
    if workerId == leaderId && workerwld >= 0.8 # && workloads[myid-1] < 0.8
       # myid = myid()
            
        csId[myid-1] = myid
        cswld[myid-1] = workloads[myid-1]
        nextId = myid+1
        if nextId > length(workers())+1
            nextId = 2
        end
        println("election started: $(cswld[myid-1])")
        election(csId[myid-1], cswld[myid-1])
    end
    #end
end


@everywhere function workers_init()
    #global workloads
    while true
        workload = round(rand(), digits = 1)
        id = myid()
        workloads[id-1] = workload
        
        for w in workers()
            if w != id
                @spawnat w updateWorkloads(id,workloads[id-1])
            end
        end
        println("Workloads: $(workloads)")
                            
        sleep(2)
    end
end

# init
for w in workers()
    @spawnat w workers_init()   
end


  Updating registry at `/srv/julia/pkg/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
      From worker 4:	Workloads: [0.0 0.0 0.2]
      From worker 2:	Workloads: [0.0 0.0 0.2]
      From worker 3:	Workloads: [0.0 0.0 0.2]
      From worker 4:	Workloads: [0.0 0.0 0.6]
      From worker 2:	Workloads: [0.4 0.0 0.6]
      From worker 3:	Workloads: [0.4 0.4 0.6]
      From worker 4:	Workloads: [0.4 0.4 0.7]
      From worker 2:	Workloads: [0.3 0.4 0.7]
      From worker 3:	Workloads: [0.3 0.1 0.7]
      From worker 4:	Workloads: [0.3 0.1 0.8]
      From worker 2:	Workloads: [0.2 0.1 0.8]
      From worker 3:	Workloads: [0.2 0.4 0.8]
      From worker 4:	Workloads: [0.2 0.4 0.8]
      From worker 2:	Workloads: [0.1 0.4 0.8]
      From worker 3:	Workloads: [0.1 0.5 0.8]
      From worker 4:	Workloads: [0.1 0.5 0.4]
      From worker 2:


## Entrega

As respostas dever�o ser entregues na parte indicada desse arquivo reservada ao c�digo fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programa��o [**Julia** vers�o 1.1.0](https://julialang.org/downloads/). Se preferir, utilize a [JuliaBox](https://juliabox.com) para fazer seu trabalho.

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. N�o se esque�a de **testar seu arquivo `.ipynb`** antes de envi�-lo.


### Forma 

O arquivo IJulia Notebook dever� ser entregue ao Professor **exclusivamente atrav�s do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho � exclusivamente dos discentes. Ser�o ignorados os trabalhos cujos arquivos n�o conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega est� descrito no **Google Classroom**.


### Pl�gio

A nota zero ser� atribu�da caso haja qualquer tipo de c�pia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre pl�gio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. Andr� Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e est� licenciado sob a licen�a _GNU General Public License v3.0_.